In [9]:
!pip install --upgrade focalnet-tensorflow 


import tensorflow as tf
from tensorflow import keras
from focalnet_tensorflow import focalnet_tiny_iso_16
import tensorflow_addons as tfa
import tensorflow_datasets as tfds


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
train_ds, validation_ds = tfds.load(
    "tf_flowers",
    split=["train[:85%]", "train[85%:]"],
    as_supervised=True,
)

In [11]:
RESIZE_TO = 384
CROP_TO = 224
BATCH_SIZE = 64
STEPS_PER_EPOCH = 10
AUTO = tf.data.AUTOTUNE  # optimise the pipeline performance
NUM_CLASSES = 5  # number of classes
SCHEDULE_LENGTH = (
    500  # we will train on lower resolution images and will still attain good results
)
SCHEDULE_BOUNDARIES = [
    200,
    300,
    400,
]  # more the dataset size the schedule length increase
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
num_epochs = 100
num_classes = 5  # number of classes

In [21]:
@tf.function
def preprocess_train(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.resize(image, (RESIZE_TO, RESIZE_TO))
    image = tf.image.random_crop(image, (CROP_TO, CROP_TO, 3))
    image = image / 255.0
    return (image, label)


@tf.function
def preprocess_test(image, label):
    image = tf.image.resize(image, (CROP_TO, CROP_TO))
    image = image / 255.0
    return (image, label)


In [22]:
# Training pipeline
pipeline_train = (
    train_ds.shuffle(10000)
    # .repeat(repeat_count)  # Repeat dataset_size / num_steps
    .map(preprocess_train, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

# Validation pipeline
pipeline_validation = (
    validation_ds.map(preprocess_test, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

In [23]:
model = focalnet_tiny_iso_16(num_classes = num_classes)

In [24]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_141 (Conv2D)            (None, 14, 14, 192)  147648      ['input_4[0][0]']                
                                                                                                  
 reshape_159 (Reshape)          (None, 196, 192)     0           ['conv2d_141[0][0]']             
                                                                                                  
 layer_normalization_84 (LayerN  (None, 196, 192)    384         ['reshape_159[0][0]']      

In [ ]:
optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )
model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            # keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )
history = model.fit(
    pipeline_train,
    batch_size=BATCH_SIZE,
    epochs=1,
    # steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=pipeline_validation,
    # callbacks=train_callbacks,
)